# Witaj w generatorze checklist żeglarskich!

In [ ]:
import os

if not os.path.exists('sailing-checklist'):
    !git clone https://github.com/SailNCode/sailing-checklist.git

In [ ]:
# @title
%pip install -q ipywidgets

Wybierz charakter rejsu (można kilka)

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display

types = {
    "mazury": "Mazury",
    "warm_sea": "Ciepłe morze"
    # "cold_see": "Chłodne morze",
    # "winter_cruise": "Zimowy rejs",
    # "foreign": "Zagraniczny",
    # "bojery": "Bojery"
}
checkboxes = {}

for key, value in types.items():
  checkboxes[key] = widgets.Checkbox(description=value);
display(*checkboxes.values());





Podaj prefiks dla każdej kategorii (opcjonalne)


In [ ]:
# @title
category_prefix_input = widgets.Text(
    value='',
    placeholder='wpisz prefix'
    # description='Uwagi:'
)
display(category_prefix_input)

Podaj prefiks dla każdego elementu listy (opcjonalne)

In [ ]:
# @title
element_prefix_input = widgets.Text(
    value='',
    placeholder='wpisz prefix'
    # description='Uwagi:'
)
display(element_prefix_input)

Wygeneruj checklistę żeglarską poniżej

In [ ]:
# @title
import json
import os
from pathlib import Path
from IPython.display import Javascript

directory = "sailing-checklist/categories"
items_lists = {}

button = widgets.Button(description="Wygeneruj")
def on_button_click(b):
  cagtegory_prefix = category_prefix_input.value.strip()
  element_prefix = element_prefix_input.value.strip()
  joined_list = {};

  chosen_categories = ["always_get"]
  for key, value in checkboxes.items():
    if value.value:
      chosen_categories.append(key)
  for category in chosen_categories:
    filename = category+".json";
    try:
      with open(Path(directory, filename), 'r') as file:
        data = json.load(file)
        items_lists[category]=data;

    except FileNotFoundError:
        print(f"Error: There is no {category} list.")
        continue;

    for name, item in items_lists[category].items():
        if (joined_list.get(name) == None):
            joined_list[name] = [];
        if isinstance(item, str):
          if item not in joined_list[name]:
            joined_list[name].append(item)
        if isinstance(item, list):
          joined_list[name] = list(dict.fromkeys(joined_list[name] + item))
  output = ""
  for key, value in joined_list.items():
    output += f"{cagtegory_prefix}{key}\n"
    for item in value:
      output += f"\t{element_prefix}{item}\n"
  if output.endswith('\n'):
    output = output[:-1]
  text_output.value = output;

button.on_click(on_button_click)

# display(button)
text_output = widgets.Textarea(
    value='',
    placeholder='Text will appear here...',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)
copy_button = widgets.Button(
    description='Copy',
    button_style='success',
    icon='copy'
)

# Copy function
def on_copy_clicked(b):
    display(Javascript(f'''
        navigator.clipboard.writeText(`{text_output.value}`);
    '''))

copy_button.on_click(on_copy_clicked)

display(widgets.VBox([text_output, widgets.HBox([button, copy_button])]))